In [1]:
import json
import tweepy
import numpy as np
import os

# BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAGvGbQEAAAAACMRCC9dW75oHLdZY872KQV%2FwncY%3DAYzORvis40n3ndPKneBisKPBQ4sYVpNthL0YHJLJZ9d6WmAjWY'
# BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAAoxbAEAAAAApib47uKVACiyXfOSAj8BELeJaf4%3DT4L0eLVTNooQRZVftkppGGnEoRqau3K2R73ZJYo3jB8TZFghZJ'
# BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAGjcbAEAAAAAnAt%2FNMSgejEX0q4HaxMGYfwExWU%3DT4mLNCEmm8oJxrCYbGGYDzEvuFwHPr4VHYvrzXJooQLTyeAbe2'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAACvAbgEAAAAAZTSFcdm27S5RfVV%2BCjCzRgcI1Ko%3DVJV80daHMDeiwLxxb9kxCWUZthIjLLFQ3FwCFXdBaNTcdILng7'

In [2]:
def authenticate(BEARER_TOKEN):

    client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=True)

    return client

client = authenticate(BEARER_TOKEN)

In [3]:

def fetch_tweet(client, id):

    tweet = {}
    response = client.get_tweets(id, 
                                tweet_fields = ['attachments', 'created_at', 'entities', 'geo', 
                                                'public_metrics', 'referenced_tweets', 
                                                'author_id', 'source', 'context_annotations', 'in_reply_to_user_id', 'lang'],
                                place_fields = ['geo', 'country', 'name', 'place_type'],
                                user_fields = ['created_at', 'description','entities', 'public_metrics', 'verified'],
                                media_fields = ['url', 'duration_ms', 'public_metrics', 'alt_text'],
                                expansions = 'geo.place_id,author_id,attachments.media_keys,in_reply_to_user_id')
    try:
        if response.data is not None:
            for t in response.data:
                tweet.update(t.data)
    except Exception as e:
        print(f'Exception for tweet: {e}')
        pass
    try:
        if response.includes.get('users', None) is not None:   
            for user in response.includes['users']:
                tweet['user'] = user.data
    except Exception as e:
        print(f'Exception for users: {e}')
        pass
    try:
        if response.includes.get('place', None) is not None:
            for place in response.includes['place']:
                tweet.update(place.data)
    except Exception as e:
        print(f'Exception for place: {e}')
        pass
    try:
        if response.includes.get('media', None) is not None:
            for media in response.includes['media']:
                tweet.update(media.data)
    except Exception as e:
        print('f Exception for media: {e}')
        pass

    return tweet

In [4]:
def write_to_file(id, tweet, path):
    with open(path + id +'.txt', 'w') as f:
        json.dump(tweet, f)

In [5]:
def tweets_processed(filepath):

    ids = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.split(' : ')
            ids.append(line[0])
    return ids

In [6]:
def read_process(client, r_path, w_path, logpath):

    ids = tweets_processed(logpath)   
     
    with open(r_path, 'r', encoding = 'utf-8') as f:
        for line in f:
            line = line.strip('\n')
            line = line.split(',')
            for id in line:
                if id not in ids:
                    try:
                    
                        tweet = fetch_tweet(client, id)   

                        write_to_file(id, tweet, w_path)

                        log = id + ' : yes\n'
                        with open(logpath, 'a') as f:
                            f.write(log)  
                    except Exception as e:
                        print(f'Exception for fetch and write: {e}')
                        print(id)
                        pass
                    

In [6]:
read_process(client, 'train.data.txt', 'tweet-objects-train/', 'log.txt')

In [7]:
def saved (log_path):

    saved = []
    with open(log_path, 'r') as f:
        for line in f:
            line = line.split(' : ')
            saved.append(line[0])
    return saved

In [8]:
def all (data_path):
    ids = []
    with open(data_path, 'r', encoding = 'utf-8') as f:
        for line in f:
            line = line.strip('\n')
            line = line.split(',')
            for id in line:
                ids.append(id)
    return ids

In [9]:
data_path = 'train.data.txt'
log_path = 'log.txt'

log = saved(log_path)
total = all(data_path)

In [10]:
if len(np.unique(log)) != len(np.unique(total)):
    for id in total:
        if id not in log:
            try:
                tweet = fetch_tweet(client, id)   
                write_to_file(id, tweet, 'tweet-objects-train/')
                log = id + ' : yes\n'
                with open('log.txt', 'a') as f:
                    f.write(log)  
            except Exception as e:
                print(f'Exception for fetch and write: {e}')
                print(id)
                pass

In [11]:
check = []

path = '/Users/weimin/Downloads/NLP/project-data/data_obj/train/tweet-objects-train'

os.chdir(path)

def read_text_file(file_path):
    with open (file_path, 'r') as f:
        if f.read() == '{}':
            f_name = file_path.split('/')
            name = f_name[-1].split('.')
            check.append(name[0])

for file in os.listdir():
    if file.endswith('.txt'):
        file_path = f'{path}/{file}'
        read_text_file(file_path)

In [12]:
print(len(check))

8262


In [14]:
path = '/Users/weimin/Downloads/NLP/project-data/data_collection/crawler'

os.chdir(path)

checked = []
with open('check-log.txt', 'r') as f:
    for line in f:
        line = line.split(' : ')
        checked.append(line[0])

print(len(checked))

8305


In [13]:
for id in check:
    if id not in checked:
        try: 
            tweet = fetch_tweet(client, id)   
            if tweet != {}:
                print(tweet)
                print(id)

                write_to_file(id, tweet, 'tweet-objects-train/')

            elif tweet == {}:
                log = id + ' : yes\n'
                with open('check-log.txt', 'a') as f:
                    f.write(log)  
        except Exception as e:
            print(f'Exception for fetch and write: {e}')
            print(id)
            pass

Rate limit exceeded. Sleeping for 400 seconds.
